In [2]:
import pandas as pd
import numpy as np
from pycaret.classification import *

In [3]:
df = pd.read_parquet("../data/final/new_df.parquet")
df = df[~df["display_state"].isin([3])]
# 'deleted' iff did_self_destruct 
df.drop(columns=['flags.did_self_destruct'], inplace=True)

# Remove collections with fewer than MIN_HOLDER_THRESHOLD
df = df[df["display_stats.holders"] >= 5]
df["display_state"].replace(
    [4, 5],
    [3, 4],
    inplace=True,
)

df.to_parquet("../data/final/evree.parquet")

model = load_model("../models/final_model_preprocessed")
model

Transformation Pipeline and Model Successfully Loaded


Pipeline(steps=[('dtypes',
                 DataTypes_Auto_infer(ml_usecase='classification',
                                      target='display_state')),
                ('imputer',
                 Iterative_Imputer(classifier=LGBMClassifier(random_state=123),
                                   max_iter=5, ordinal_columns=[],
                                   random_state=123,
                                   regressor=LGBMRegressor(random_state=123),
                                   target='display_state')),
                ('new_levels1',
                 New_Catagorical_Levels_in_TestData(replacement_s...
                ('fix_perfect', Remove_100(target='display_state')),
                ('clean_names', Clean_Colum_Names()),
                ('feature_select', 'passthrough'),
                ('fix_multi',
                 Fix_multicollinearity(correlation_with_target_preference=None,
                                       target_variable='display_state',
                  

In [7]:
# plotly pie chart
from plotly import express as px
def plot_pie(df, title="DTTD NFT classification", col='display_state'):
    legend_map = {0: 'safe', 1: 'normal', 2: 'hidden', 3: 'caution', 4: 'suspicious'}
    # df['display_state_name'] = df['display_state'].apply(lambda x: legend_map[x])

    target_dist = df[col].value_counts().sort_index()
    fig = px.pie(target_dist, 
        values=col, 
        names=legend_map.values(), 
        title=title,
        labels=legend_map)
    # 800x800 fig size
    fig.update_layout(
        autosize=False,
        width=500,
        height=500,
        template="plotly_dark",
        title_font_size=20,
        title_x=0.1,
    )
    # update legend
    return fig

In [8]:
plot_pie(df, title="DTTD NFT classification", col='display_state')

In [11]:
prediction = predict_model(model, data=df)

In [12]:
pred = prediction[['contract_address', 'chain_id', 'display_state', 'Label', 'Score']]

In [16]:
plot_pie(prediction, title="Model classification", col='Label')

In [28]:
pred.to_csv("../data/predictions/evree.csv", index=False)

In [30]:
contract_address = '0xf220db48f0d3ca8a9833e0353e7497dbceae7ac6'
pred.loc[pred['contract_address'] == contract_address]

,contract_address,chain_id,display_state,Label,Score
2,0xf220db48f0d3ca8a9833e0353e7497dbceae7ac6,eth_main,1,1,0.5706


In [4]:
dashboard(model, display_format='dash')

NameError: name '_ml_usecase' is not defined